In [22]:
import numpy as np
import matplotlib.pyplot as plt

# Generate the particles

The particles are generated here. There are also checks for uniformity

In [23]:
def genParticles(L, Np, Nc, l, rng, verify=False):
    Nc = Nc
    l = l
    rints = rng.integers(low=0, high=10, size=3)

    radius = L/4
    centre = np.array([L/2, L/2, L/2])

    particles = np.zeros((Np, 9))
    sph_field = np.zeros((Np, 3))

    for i in range(Np):
        rad = radius*((4*rng.random()*radius)**(1/4)) #radius Jacobian
        thet = np.arccos(1 - 2*rng.random()) # The random for theta Jacobian
        phi = 2*np.pi*rng.random()
        sph_field[i, 0] = rad
        sph_field[i, 1] = thet
        sph_field[i, 2] = phi
        jacobian = 1
        particles[i, 0] = (rad*np.sin(thet)*np.cos(phi) + centre[0])*jacobian
        particles[i, 1] = (rad*np.sin(thet)*np.sin(phi) + centre[1])*jacobian
        particles[i, 2] = (rad*np.cos(thet) + centre[2])*jacobian

    # ----------------------------------------------
    # This is all verification down here
    # ----------------------------------------------
    if verify:
        N_block = 8
        num_in_block = np.zeros(N_block)
        for j in range(len(particles)):
            if particles[j][0] > centre[0]:
                if particles[j][1] > centre[1]:
                    if particles[j][2] > centre[2]:
                        num_in_block[0] += 1
                    else:
                        num_in_block[4] += 1
                else:
                    if particles[j][2] > centre[2]:
                        num_in_block[3] += 1
                    else:
                        num_in_block[7] += 1
            else:
                if particles[j][1] > centre[1]:
                    if particles[j][2] > centre[2]:
                        num_in_block[1] += 1
                    else:
                        num_in_block[5] += 1
                else:
                    if particles[j][2] > centre[2]:
                        num_in_block[2] += 1
                    else:
                        num_in_block[6] += 1

        radius_check = []
        Np_samples = Np//200
        for i in range(len(particles)):
            if i % Np_samples == 0:
                radius_check.append(np.sqrt(((particles[i,0] - centre[0])**2) + ((particles[i,1] - centre[1])**2) + ((particles[i,2] - centre[2])**2)))
        numline = np.zeros(len(radius_check))

        radmax = 0
        for i in range(len(radius_check)):
            if np.sqrt(((particles[i,0] - centre[0])**2) + ((particles[i,1] - centre[1])**2) + ((particles[i,2] - centre[2])**2)) > radmax:
                radmax = np.sqrt(((particles[i,0] - centre[0])**2) + ((particles[i,1] - centre[1])**2) + ((particles[i,2] - centre[2])**2))

        print("Running Sanity Checks")
        print("Debug: Each Quadrant should have around " + str(Np//N_block) + " particles")
        print("Quadrants 1-8 have: ", num_in_block)
        print("\nThe radius should be no larger than " + str(radius) + ". The maximum radius is " + str(radmax))

        # Uniformity check

        N_div = 50
        xc = np.zeros(N_div)
        a = L/2
        b = (L/2) + L/2
        for i in range(1,N_div):
            for j in range(len(particles)):
                if (particles[j, 0] > ((radius*((i-1)/N_div)) + a)) and (particles[j, 0] < ((radius*((i)/N_div)) + a)):
                    xc[i-1] += 1

        print("\nif the distribution is uniform, all the values of this matrix should be about the same:")
        print(xc)

        print("\nCheck to see if the following circle is Equally distributed")

        plt.plot(particles[:,0], particles[:,1], 'o', markersize=0.25)
        plt.title("Slice of the sphere down the middle")
        plt.gca().set_aspect('equal')
        plt.show()
    return particles




# Cartesian Mesh

Determine i,j,k

In [58]:
def calc_indices(field, l, Nc=128, L=1, verify=False):
    

    

    Ntest = len(field)
    p_set = np.zeros((Ntest, 9))

    for i in range(Ntest):
        p_set[i] = field[i]

    for i in range(len(p_set)):
        if p_set[i, 0] <= l or p_set[i, 1] <= l or p_set[i, 2] <= l or p_set[i, 0] >= L-l or p_set[i, 1] >= L-l or p_set[i, 2] >= L-l:
            p_set = np.delete(p_set, i, 0)


    lvals = np.linspace(0, L, 128, endpoint=True)
    start_indices = np.zeros(10, int)

    for i in range(1, len(start_indices)):
        for j in range(128):
            if start_indices[i] == 0 and int((lvals[j] - int(lvals[j])) * 10) == i:
                start_indices[i] = j
            else:
                continue

    x_l = np.zeros(len(p_set), int)

    dimension = 3
    N_temp = len(p_set)

    indices = np.zeros((N_temp, 6), int)

    for i in range(len(p_set)):
        xval = p_set[i,0]
        j = start_indices[int((xval - int(xval)) * 10)]
        while j < 128+1:
            if xval >= lvals[j-1] and xval <= lvals[j]:
                if xval <= lvals[j-1] + (l/2):
                    indices[i][0] = j-2
                    indices[i][1] = j-1
                else:
                    indices[i][0] = j-1
                    indices[i][1] = j
                break
            else:
                j += 1


    for i in range(len(p_set)):
        yval = p_set[i,1]
        j = start_indices[int((yval - int(yval)) * 10)]
        while j < 128+1:
            if yval >= lvals[j-1] and yval <= lvals[j]:
                if yval <= lvals[j-1] + (l/2):
                    indices[i][2] = j-2
                    indices[i][3] = j-1
                else:
                    indices[i][2] = j-1
                    indices[i][3] = j
                break
            else:
                j += 1

    for i in range(len(p_set)):
        zval = p_set[i,2]
        j = start_indices[int((zval - int(zval)) * 10)]
        while j < 128+1:
            if zval >= lvals[j-1] and zval <= lvals[j]:
                if zval <= lvals[j-1] + (l/2):
                    indices[i][4] = j-2
                    indices[i][5] = j-1
                else:
                    indices[i][4] = j-1
                    indices[i][5] = j
                break
            else:
                j += 1
    if verify:
        print("Nc", Nc, "l", l)
        print("Matrix of indices created with shape:", np.shape(indices))

    return indices, lvals
#indices, p_set, lvals = calc_indices(field)

Calculate the overlap

In [25]:
# Calculates the overlap of each particle with each cell
def overlap(p_set, l, lvals, indices, verify=False):
    dvals = np.zeros((len(p_set), 6)) # Delta parameter values
    for i in range(len(p_set)):
        pos = np.array([p_set[i][0], p_set[i][1], p_set[i][2]])
        left = np.array([p_set[i][0]-(l/2),p_set[i][1]-(l/2),p_set[i][2]-(l/2)])
        for j in range(len(pos)):
            dval = lvals[indices[i][2*j + 1]] - left[j]
            dvals[i][2*j] = dval
            dvals[i][2*j+1] = l - dval

    if verify:
        ind = 2 # Change this to the particle number
        axes = 0 # Change this to axis you want; x,y,z = 0,1,2

        x0 = p_set[ind][axes]
        lx0 = x0 - (l/2)
        rx = x0+(l/2)
        l10 = lvals[indices[ind][2*axes]]
        l20 = lvals[indices[ind][2*axes + 1]]

        print("Left side distribution:", -lx0 + l20)
        print("Right side distribution:", l - (-lx0 + l20))
        print("Sum is " + str(-lx0 + l20 + l - (-lx0 + l20)))
        print("l is " + str(l))

        plt.plot(x0, 0, 'o')
        plt.annotate(str(round(x0, 4)), xy=(x0+(l*0.05),0))
        plt.plot([lx0, rx], [0.1, 0.1], 'b-')
        plt.annotate("L/Nc = " +str(l), xy=(x0-(l*0.35), 0.2))
        plt.plot([l10, l20], [0.1,0.1], 'o')
        plt.annotate("Grid point " + str(indices[ind][2*axes]), xy=(l10-(l*0.25), -0.3))
        plt.annotate("Grid point " + str(indices[ind][2*axes + 1]), xy=(l20-(l*0.25), -0.3))
        plt.annotate("pos: "+str(round(l10, 4)), xy=(l10-(l*0.25), -0.4))
        plt.annotate("pos: "+str(round(l20, 4)), xy=(l20-(l*0.25), -0.4))
        plt.plot([lx0, l20],[-0.1,-0.1], 'y-')
        plt.plot([l20, rx], [-0.1,-0.1], 'k-')
        plt.annotate("LHS: " + str(round(-lx0 + l20, 5)), xy=(lx0-(0.2*l), -0.5))
        plt.annotate("RHS: " + str(round(l - (-lx0 + l20), 5)), xy=(rx-(0.2*l), -0.5))
        plt.xlim(x0-(2*l), x0+l)
        plt.ylim(-1,1)
        plt.yticks([])
        plt.title("Mass Distribution of point " + str(ind) + " on axis " + str(axes))
        plt.xlabel("Distance on axis " + str(axes) + " (m)")
        plt.show()
    
    return dvals

In [26]:
#dvals = overlap(p_set, indices)
#print("Delta param values created", np.shape(dvals))

Add to density matrix

In [29]:
def dens_overlap(dvals,indices,l,Nc=128,Np=32**3, mp=1, verify=False):
    density = np.zeros((Nc,Nc,Nc))
    lcube = l**3

    for i in range(len(dvals)):
        i1, i2, j1, j2, k1, k2 = indices[i]
        density[i1,j1,k1] += (mp*dvals[i][0]*dvals[i][2]*dvals[i][4])/lcube
        density[i1,j2,k1] += (mp*dvals[i][0]*dvals[i][3]*dvals[i][4])/lcube
        density[i1,j1,k2] += (mp*dvals[i][0]*dvals[i][2]*dvals[i][5])/lcube
        density[i1,j2,k2] += (mp*dvals[i][0]*dvals[i][3]*dvals[i][5])/lcube
        density[i2,j1,k1] += (mp*dvals[i][1]*dvals[i][2]*dvals[i][4])/lcube
        density[i2,j2,k1] += (mp*dvals[i][1]*dvals[i][3]*dvals[i][4])/lcube
        density[i2,j1,k2] += (mp*dvals[i][1]*dvals[i][2]*dvals[i][5])/lcube
        density[i2,j2,k2] += (mp*dvals[i][1]*dvals[i][3]*dvals[i][5])/lcube
    
    density = density/lcube
    
    if verify:
        print("Density matrix created with particle mass", mp, ", matrix size", np.shape(density))
        massval = 0
        for i in range(Nc):
            for j in range(Nc):
                for k in range(Nc):
                    massval += density[i,j,k]

        ave = massval/(Nc**3)
        print("Summing all density values and dividing by volume yields", ave)
        print("This value should be the same as", Np*mp)

    return density

#mp = 1
#l3 = l**3

#print("Density matrix created with particle mass", mp, ", matrix size", np.shape(density))


Verification

In [7]:
def kvec(Nk):
    half = Nk//2
    kx = np.zeros(Nk)
    for i in range(Nk):
        if i <= half:
            kx[i] = (2*np.pi/Nk)*i
        else:
            kx[i] = (2*np.pi/Nk)*(i - Nk)
    return np.array(kx)
print("Function to calculate k values created")


Function to calculate k values created


In [8]:
def w(k, G=1):
    kx = k[0]
    ky = k[1]
    kz = k[2]
    return -(4*np.pi*G)*(1/(((2*np.sin(kx/2))**2) + ((2*np.sin(ky/2))**2) + ((2*np.sin(kz/2))**2)))
print("Function to calculate w(k) created, with adjustable parameter G")

Function to calculate w(k) created, with adjustable parameter G


In [9]:
def create_k_values(Nc, verify=False):
    kvals = kvec(Nc)
    if verify:
        print("List of k values created. 5 equally spaced samples:", kvals[0], kvals[len(kvals)//4], kvals[len(kvals)//2], kvals[3*(len(kvals)//4)], kvals[-1])
    return kvals

In [10]:

def create_w_k(Nc, kvals, G=1, verify=False):
    wk = np.zeros((Nc, Nc, Nc))
    for i in range(Nc):
        for j in range(Nc):
            for k in range(Nc):
                wk[i,j,k] = w(np.array([kvals[i], kvals[j], kvals[k]]), G=G)
        if verify:
            if i % (128//10) == 0:
                print((i/Nc)*100, "%")
    wk[0,0,0] = 0.0
    
    if verify:
        print("wk created with G =", G, ". Shape of matrix is", np.shape(wk))
    return wk

Creating $\textbf{w(k)}$

$\rho(\textbf{k}) = \textrm{FFT}(\rho(\phi))$

In [68]:
def FFT_density(density, method="forward", fftn_val=True, verify=False):
    rho = np.copy(density)
    if fftn_val:
        density_k = np.fft.fftn(rho, norm=method)
    else:
        density_k = np.fft.rfftn(rho, norm=method)

    if verify:
        print("rho(k) created, using", method, "FFT. Shape is", np.shape(density_k))
    return density_k

$\Phi(\textbf{k}) = \rho(\textbf{k}) w(\textbf{k})$

In [72]:
def calculate_phi_k(density_k,wk,Nc=128,verify=False, slice=False):
    Phi_k = np.zeros((Nc, Nc, Nc), complex)
    if slice:
        wk = wk[:,:,:density_k.shape[2]]

    for i in range(len(density_k)):
        for j in range(len(density_k[0])):
            for k in range(len(density_k[0][0])):
                Phi_k[i,j,k] = density_k[i,j,k]*wk[i,j,k]
        if verify:
            if i % (128//10) == 0:
                print((i/len(density_k))*100, "%")
    if verify:
        print("Phi_k is done. With a size", np.shape(Phi_k))
    return Phi_k

Calculating the FFT Inverse of $\Phi(\textbf{k})$

In [76]:
def phi_k_to_phi_x(Phi_k, L=1.0, Nc=128, verify=False, mult_factor=0.0):
    Potential = np.fft.irfftn(Phi_k, s=(128,128,128))*((L/Nc)**mult_factor)
    if verify:
        plt.imshow(Potential[0])
        plt.colorbar().set_label("$\Phi(x)$")
        plt.xticks([])
        plt.yticks([])
        plt.set_cmap("hot")
        plt.title("Slice of potential as Sanity Check")
        plt.show()
        print("Potential Phi(x) created with shape", np.shape(Potential))
    return Potential

Calculating Force

In [14]:
def forward_diff(f_x, f_xph, h=1e-8):
    return (f_xph - f_x)/h

def backward_diff(f_x, f_xmh, h=1e-8):
    return (f_x - f_xmh)/h

def central_diff(f_xmh, f_xph, h=1e-8):
    return (f_xph - f_xmh)/(2*h)

def test_func(x):
    return 3*(x**5) - 4*(x**4)

def verify_derivative_functions(a=0,b=1.5,Ntt=50, details=False):
    testy = np.zeros(Ntt)
    xtest = np.linspace(a, b, Ntt)
    hval = (b-a)/Ntt
    for i in range(len(testy)):
        testy[i] = test_func(xtest[i])

    dtesty = np.zeros(Ntt)
    for i in range(len(dtesty)):
        if i == 0:
            dtesty[i] = forward_diff(testy[i], testy[i+1], h=hval)
        elif i == len(dtesty) - 1:
            dtesty[i] = backward_diff(testy[i], testy[i-1], h=hval)
        else:
            dtesty[i] = central_diff(testy[i-1], testy[i+1], h=hval)

    if details:
        print("initial matrix", testy)
        print("derivatives", dtesty)

    plt.plot(xtest, testy)
    plt.plot(xtest, dtesty, 'k-')
    plt.xlim(0, 1.5)
    plt.ylim(-2.5,2)
    plt.show()

In [142]:

# Remnants from just testing out the mechanism
def calculate_force(Potential,l,Nc=128,verify=False):
    N_test = Nc
    test_array = Potential
    hval = l

    force_array = np.zeros((N_test, N_test, N_test, 3), float)

    diff_functions = [central_diff, backward_diff, forward_diff]

    axes = 2

    for i in range(len(test_array)):
        for j in range(len(test_array)):
            for k in range(len(test_array)):
                diff_method = np.zeros(axes+1, int)
                vals = np.zeros(6)
                if i == 0:
                    diff_method[0] = 2
                    vals[0] = test_array[i,j,k]
                    vals[1] = test_array[i+1,j,k]
                elif i == len(test_array) - 1:
                    diff_method[0] = 1
                    vals[0] = test_array[i,j,k]
                    vals[1] = test_array[i-1,j,k]
                else:
                    vals[0] = test_array[i-1,j,k]
                    vals[1] = test_array[i+1,j,k]

                if j == 0:
                    diff_method[1] = 2
                    vals[2] = test_array[i,j,k]
                    vals[3] = test_array[i,j+1,k]
                elif j == len(test_array) - 1:
                    diff_method[1] = 1
                    vals[2] = test_array[i,j,k]
                    vals[3] = test_array[i,j-1,k]
                else:
                    vals[2] = test_array[i,j-1,k]
                    vals[3] = test_array[i,j+1,k]

                if k == 0:
                    diff_method[2] = 2
                    vals[4] = test_array[i,j,k]
                    vals[5] = test_array[i,j,k+1]
                elif k == len(test_array) - 1:
                    diff_method[2] = 1
                    vals[4] = test_array[i,j,k]
                    vals[5] = test_array[i,j,k-1]
                else:
                    vals[4] = test_array[i,j,k-1]
                    vals[5] = test_array[i,j,k+1]

                force_array[i,j,k,0] = diff_functions[diff_method[0]](vals[0], vals[1], h=hval)
                force_array[i,j,k,1] = diff_functions[diff_method[1]](vals[2], vals[3], h=hval)
                force_array[i,j,k,2] = diff_functions[diff_method[2]](vals[4], vals[5], h=hval)
    if verify:
        print("Force Array Successfully Created with shape", np.shape(force_array))
        #print(np.shape(force_array[:,:,0,0]))
        f_array = np.zeros(Nc)
        for i in range(Nc):
            for j in range(Nc):
                f_array[i] += force_array[j, i, 64, 0]
        plt.plot(np.linspace(0.54, 1, Nc//2), f_array[64:], 'o')
        plt.plot(np.linspace(0.54, 1, Nc), 1/((np.linspace(0.54, 1, Nc) - 0.5)**2))
        plt.show()
    return force_array       
    


Calculate the field with the force

In [16]:
def force_overlap(dvals, indices, force,l, mp=1, Nc=128):
    density = np.zeros((Nc,Nc,Nc, 3))
    coeffs = []
    lcube = l**3
    #print(dvals[0])
    for i in range(len(dvals)):
        i1, i2, j1, j2, k1, k2 = indices[i]
        #print(force[dvals[i][0],dvals[i][2],dvals[i][4],:])

        density[i1,j1,k1] += force[i1,j1,k1,:]*(mp*dvals[i][0]*dvals[i][2]*dvals[i][4])/lcube
        density[i1,j2,k1] += force[i1,j2,k1,:]*(mp*dvals[i][0]*dvals[i][3]*dvals[i][4])/lcube
        density[i1,j1,k2] += force[i1,j1,k2,:]*(mp*dvals[i][0]*dvals[i][2]*dvals[i][5])/lcube
        density[i1,j2,k2] += force[i1,j2,k2,:]*(mp*dvals[i][0]*dvals[i][3]*dvals[i][5])/lcube
        density[i2,j1,k1] += force[i2,j1,k1,:]*(mp*dvals[i][1]*dvals[i][2]*dvals[i][4])/lcube
        density[i2,j2,k1] += force[i2,j2,k1,:]*(mp*dvals[i][1]*dvals[i][3]*dvals[i][4])/lcube
        density[i2,j1,k2] += force[i2,j1,k2,:]*(mp*dvals[i][1]*dvals[i][2]*dvals[i][5])/lcube
        density[i2,j2,k2] += force[i2,j2,k2,:]*(mp*dvals[i][1]*dvals[i][3]*dvals[i][5])/lcube

        coeffs.append(np.array([dvals[i][0]*dvals[i][2]*dvals[i][4], dvals[i][0]*dvals[i][3]*dvals[i][4], dvals[i][0]*dvals[i][2]*dvals[i][5], dvals[i][0]*dvals[i][3]*dvals[i][5], dvals[i][1]*dvals[i][2]*dvals[i][4], dvals[i][1]*dvals[i][3]*dvals[i][4], dvals[i][1]*dvals[i][2]*dvals[i][5],dvals[i][1]*dvals[i][3]*dvals[i][5]]))
    return density, np.array(coeffs)

Add Acceleration to field

In [17]:
def add_accelerations(field, acc, coeff, indices):
    for i in range(len(field)):
        i1, i2, j1, j2, k1, k2 = indices[i]
        field[i][6] += (acc[i1, j1, k1, 0]*coeff[i,0]) + (acc[i1, j2, k1,0]*coeff[i,1]) + (acc[i1,j1,k2,0]*coeff[i,2]) + (acc[i1,j2,k2,0]*coeff[i,3]) + (acc[i2,j1,k1,0]*coeff[i,4]) + (acc[i2,j2,k1,0]*coeff[i,5]) + (acc[i2,j1,k2,0]*coeff[i,6]) + (acc[i2,j2,k2,0]*coeff[i,7])
        field[i][7] += (acc[i1, j1, k1, 1]*coeff[i,0]) + (acc[i1, j2, k1,1]*coeff[i,1]) + (acc[i1,j1,k2,1]*coeff[i,2]) + (acc[i1,j2,k2,1]*coeff[i,3]) + (acc[i2,j1,k1,1]*coeff[i,4]) + (acc[i2,j2,k1,1]*coeff[i,5]) + (acc[i2,j1,k2,1]*coeff[i,6]) + (acc[i2,j2,k2,1]*coeff[i,7])
        field[i][8] += (acc[i1, j1, k1, 2]*coeff[i,0]) + (acc[i1, j2, k1,2]*coeff[i,1]) + (acc[i1,j1,k2,2]*coeff[i,2]) + (acc[i1,j2,k2,2]*coeff[i,3]) + (acc[i2,j1,k1,2]*coeff[i,4]) + (acc[i2,j2,k1,2]*coeff[i,5]) + (acc[i2,j1,k2,2]*coeff[i,6]) + (acc[i2,j2,k2,2]*coeff[i,7])

In [18]:
# This function only works if velocity at t=0 is 0. In our case it is
def calc_init_timestep(acc,l, lamb=0.5):
    return lamb*np.sqrt(l/np.max(acc))

In [19]:
def calc_timestep(acc, vel, l, lamb=0.5):
    t1 = l/np.max(vel)
    t2 = np.sqrt(l/np.max(acc))
    if t1 > t2:
        return lamb*t1
    else:
        return lamb*t2

In [60]:
def calc_init_pos(particles):
    pos = np.zeros((len(particles), 3))
    for i in range(len(particles)):
        pos[i] = np.array([particles[i][0], particles[i][1], particles[i][2]])
    return pos

In [20]:
#vel = np.zeros((Nc,Nc,Nc,3))
def update_vel(acc, dt, Nc=128):
    vel = np.zeros((Nc,Nc,Nc,3))
    for i in range(len(acc)):
        for j in range(len(acc)):
            for k in range(len(acc)):
                vel[i,j,k,0] += acc[i,j,k,0]*dt
                vel[i,j,k,1] += acc[i,j,k,1]*dt
                vel[i,j,k,2] += acc[i,j,k,2]*dt
    return vel


In [95]:
def update_acceleration(particles, l, Nc=128, G=1, fft=False, slice=True, mult=1):
    particle_current = np.copy(particles)
    indices, lvals = calc_indices(particle_current, l)
    delta_x_values = overlap(particle_current, l, lvals, indices)
    density_i = dens_overlap(delta_x_values, indices, l)
    kvalues = create_k_values(Nc)
    w_k_values = create_w_k(Nc, kvalues, G)
    FFT_rho = FFT_density(density_i, fftn_val=fft)
    phi_k = calculate_phi_k(FFT_rho, w_k_values, slice=True)
    phi_x = -1 * phi_k_to_phi_x(phi_k, mult_factor=mult)
    force = calculate_force(phi_x, l)
    acceleration_field, delta_x_coefficients = force_overlap(delta_x_values, indices, force, l)
    add_accelerations(particles, acceleration_field, delta_x_coefficients, indices)
    

In [177]:
def calc_timestep_1(dt0, particles, l, Nc=128, G=1, lamb=0.5, fixed_timestep=False, max_t=1):
    snapshots = []
    if fixed_timestep:
        dt_0 = 0.05*(np.sqrt(((np.pi**2)*(0.25**3))/(4*(32**3))))
    else:
        dt_0 = dt0
    snap = 0
    while dt_0 <= (np.sqrt(((np.pi**2)*(0.25**3))/(4*(32**3)))):
        #Kick
        #Calc v_(i+1/2)
        for i in range(len(particles)):
            for j in range(3,6):
                particles[i][j] += ((0.5)*(particles[i][j+3])*(dt_0))


        #Drift
        # Calc x_(i+1)
        for i in range(len(particles)):
            for j in range(3):
                particles[i][j] += particles[i][j+3]*dt_0

        
        #Calc new acceleration
        update_acceleration(particles, l, Nc, G)

        #Kick
        #Calc v_(i+1)
        for i in range(len(particles)):
            for j in range(3,6):
                particles[i][j] += particles[i][j+3]*(dt_0/2)
        #print(particles)
        snapshots.append(particles[:, 0:3])
        if fixed_timestep:
            dt_0 += 0.05*(np.sqrt(((np.pi**2)*(0.25**3))/(4*(32**3))))
        else:
            dt_0 += calc_timestep(particles[:, 6:9], particles[:, 3:6], l, lamb=lamb)
        print((snap/20)*100, "%")
        snap += 1
        if snap > 20:
            break
    return snapshots
    # remember to update acc values after x and v

In [179]:

def repeat_for_time(dt_0, particles, l, Nc=128, G=1, lamb=0.5, details=True, max_t_steps=1):
    snapshot_times = calc_timestep_1(dt_0, particles, l, Nc=Nc, G=G, fixed_timestep=True, max_t=1, lamb=lamb)
    print("Calculated snapshots.")
    return snapshot_times

        

In [180]:
def initial_setup(L, Np, Nc, l, rng, mult_factor=0.5, G=1, lamb=0.5, details=True, max_t_steps=1, debug=False, super_debug=False):
    

    # All within t=0
    init_particle = genParticles(L, Np, Nc, l, rng, verify=super_debug)
    if details:
        print("Initial Particle Setup Completed.")

    init_indices, lvals = calc_indices(init_particle, l, verify=debug)
    if details:
        print("Initial Indices, Division Values Setup Completed.")

    delta_x_values = overlap(init_particle, l, lvals, init_indices, verify=debug)
    if details:
        print("Delta x_a, Delta y_a, Delta z_a Initially Calculated.")


    density = dens_overlap(delta_x_values, init_indices, l, verify=super_debug)
    if details:
        print("Initial Density Matrix Made.")
    kvalues = create_k_values(Nc, verify=super_debug)
    if details:
        print("k_x values created.")
    w_k_values = create_w_k(Nc, kvalues, G, verify=super_debug)
    if details:
        print("w(k) made.")
    FFT_rho = FFT_density(density, fftn_val=False, verify=super_debug)
    if details:
        print("FFT(rho) calculated.")
    phi_k = calculate_phi_k(FFT_rho, w_k_values, slice=True, verify=super_debug)
    if details:
        print("Phi(k) created.")
    phi_x = -1 * phi_k_to_phi_x(phi_k, mult_factor=mult_factor, verify=super_debug)
    if details:
        print("Inverse FFT(Phi(k)) = Phi(x) Completed.")


    force = calculate_force(phi_x, l, verify=debug)
    if details:
        print("Initial Force Field calculated.")
    
    acceleration_field, delta_x_coefficients = force_overlap(delta_x_values, init_indices, force, l) 
    if details:
        print("Initial Acceleration Field Calculated") 
     
    particles = np.copy(init_particle)
    add_accelerations(particles, acceleration_field, delta_x_coefficients, init_indices)
    if details:
        print("Added Accelerations.")

    init_positions = calc_init_pos(init_particle)
    if details:
        print("Isolated Initial Positions")

    # Now looking to t != 0
    dt0 = calc_init_timestep(acceleration_field, lamb)
    if details:
        print("Current time step calculated as", dt0)
        print("Moving to calculate for", max_t_steps, "more steps")
    
    return repeat_for_time(dt0, particles, l, Nc=128, G=1, lamb=0.5), particles



L = 1.0
Np = 32**3
Nc = 128
l = L/Nc
rng = np.random.default_rng(1234)
mult_factor = -1.875 #The value of ? in the equation (L/Nc)^? # Tent. guess is -1.875

debug = False # Change this if you want the Debug section, 4.3

super_debug = False # It's for all debugs. All extra graphs made etc.

snapshot_times, particles = initial_setup(L, Np, Nc, l, rng, mult_factor=mult_factor, debug=debug, super_debug=super_debug)

Initial Particle Setup Completed.
Initial Indices, Division Values Setup Completed.
Delta x_a, Delta y_a, Delta z_a Initially Calculated.
Initial Density Matrix Made.
k_x values created.


/tmp/ipykernel_1316/1993806342.py:5: RuntimeWarning: divide by zero encountered in scalar divide
  return -(4*np.pi*G)*(1/(((2*np.sin(kx/2))**2) + ((2*np.sin(ky/2))**2) + ((2*np.sin(kz/2))**2)))


w(k) made.
FFT(rho) calculated.
Phi(k) created.
Inverse FFT(Phi(k)) = Phi(x) Completed.


/tmp/ipykernel_1316/3629369823.py:2: DeprecationWarning: `axes` should not be `None` if `s` is not `None` (Deprecated in NumPy 2.0). In a future version of NumPy, this will raise an error and `s[i]` will correspond to the size along the transformed axis specified by `axes[i]`. To retain current behaviour, pass a sequence [0, ..., k-1] to `axes` for an array of dimension k.
  Potential = np.fft.irfftn(Phi_k, s=(128,128,128))*((L/Nc)**mult_factor)


Initial Force Field calculated.
Initial Acceleration Field Calculated
Added Accelerations.
Isolated Initial Positions
Current time step calculated as 4.366718498296189e-05
Moving to calculate for 1 more steps
0.0 %
5.0 %
10.0 %
15.0 %
20.0 %
25.0 %
30.0 %
35.0 %
40.0 %
45.0 %
50.0 %
55.00000000000001 %
60.0 %
65.0 %
70.0 %
75.0 %
80.0 %
85.0 %
90.0 %
Calculated snapshots.


In [198]:
xlines = np.zeros((Np, len(snapshot_times)))
ylines = np.zeros((Np, len(snapshot_times)))
zlines = np.zeros((Np, len(snapshot_times)))
print(len(xlines))

for i in range(len(snapshot_times)):
    for j in range(32768):
        xlines[j][i] = snapshot_times[i][j][0]
        ylines[j][i] = snapshot_times[i][j][1]
        zlines[j][i] = snapshot_times[i][j][2]




32768


In [ ]:
for i in range(1):
    plt.plot(xlines[i], ylines[i], 'r-')
    print(xlines[i], ylines[i])
plt.show()

In [ ]:
for i in range(len(ylines)):
    plt.plot(ylines[i], zlines[i], 'k-')
plt.show()

In [ ]:
magnitudes = np.zeros((len(snapshot_times), 32**3))
for i in range(len(snapshot_times)):
    for j in range(32**3):
        magnitudes[i, j] = np.linalg.norm(snapshot_times[j])

sorted = np.zeros((len(snapshot_times), 32**3))
for i in range(len(snapshot_times)):
    sorted[i] = np.sort(magnitudes[i])

plt.plot(sorted[10])
plt.show()

In [ ]:
accelerations = np.zeros((len(snapshot_times) 32**3))

for i in range(len(snapshot_times)):
    for j in range(32**3):
        accelerations[i, j] = np.linalg.norm(particles[j][6:9])
